# Copyright

<PRE>
Copyright (c) Bálint Gyires-Tóth - All Rights Reserved
You may use and modify this code for research and development purpuses.
Using this code for educational purposes (self-paced or instructor led) without the permission of the author is prohibited.
</PRE>

# Assignment: RNN text generation with your favorite book


In [1]:
import string
import numpy as np
import tensorflow as tf

## 1. Dataset
- Download your favorite book from https://www.gutenberg.org/
- Split into training (80%) and validation (20%).

In [2]:
f = open("/content/gatsby.txt", "r")
book = f.read()

train_size = int(0.8 * len(book))

train = book[:train_size]
val = book[train_size:]

## 2. Preprocessing
- Convert text to lowercase.  
- Remove punctuation (except basic sentence delimiters).  
- Tokenize by words or characters (your choice).  
- Build a vocabulary (map each unique word to an integer ID).

In [3]:
text = book.lower()  # convert text to lowercase

# remove punctuation
remove = ''.join([char for char in string.punctuation if char not in ".!?"])
translator = str.maketrans('', '', remove)
text = text.translate(translator)
text = ' '.join(text.split())  # removes white space

# tokenize
vocab = sorted(set(list(text)))
chars = tf.strings.unicode_split(text, input_encoding='UTF-8')

# vocabulary
chars_to_ids = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
ids = chars_to_ids(chars)

# split preprocessed data into training and validation
train = ids[:train_size]
val = ids[train_size:]
print(f"Train size: {len(train)}")
print(f"Validation size: {len(val)}")

Train size: 232061
Validation size: 49744


## 3. Embedding Layer in Keras
Below is a minimal example of defining an `Embedding` layer:
```python
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    input_dim=vocab_size,     # size of the vocabulary
    output_dim=128,           # embedding vector dimension
    input_length=sequence_length
)
```
- This layer transforms integer-encoded sequences (word IDs) into dense vector embeddings.

- Feed these embeddings into your LSTM or GRU OR 1D CNN layer.

In [4]:
# function that splits sequences into first 99 characters (x) and last character (y)
def sequence(text,sequence_length):
  x = []; y = []
  for i in range(0,len(text)-sequence_length,1):
    x.append(text[i:i+sequence_length])
    y.append(text[i+sequence_length])
  return np.array(x), np.array(y)

sequence_length = 100
x_train, y_train = sequence(train,sequence_length)
x_val, y_val = sequence(val,sequence_length)

In [5]:
print(x_train.shape,y_train.shape)
print(x_val.shape,y_val.shape)

(231961, 100) (231961,)
(49644, 100) (49644,)


In [6]:
vocab_size = len(chars_to_ids.get_vocabulary())
print(vocab_size)

54


## 4. Model
- Implement an LSTM or GRU or 1D CNN-based language model with:
  - **The Embedding layer** as input.
  - At least **one recurrent layer** (e.g., `LSTM(256)` or `GRU(256)` or your custom 1D CNN).
  - A **Dense** output layer with **softmax** activation for word prediction.
- Train for about **5–10 epochs** so it can finish in approximately **2 hours** on a standard machine.


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
model = Sequential()
model.add(Embedding(
    input_dim=vocab_size,     # size of the vocabulary
    output_dim=128,           # embedding vector dimension
    input_length=sequence_length
))
model.add(LSTM(256, return_sequences=False))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


## 5. Training & Evaluation
- **Monitor** the loss on both training and validation sets.
- **Perplexity**: a common metric for language models.
  - It is the exponent of the average negative log-likelihood.
  - If your model outputs cross-entropy loss `H`, then `perplexity = e^H`.
  - Try to keep the validation perplexity **under 50** if possible. If you have higher value (which is possible) try to draw conclusions, why doesn't it decrease to a lower value.

In [9]:
network_history = model.fit(x_train, y_train,
                            validation_data=(x_val, y_val),
                            batch_size=128,
                            epochs=5,
                            verbose=1)

Epoch 1/5
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 1733s 955ms/step - accuracy: 0.3126 - loss: 2.3910 - val_accuracy: 0.4168 - val_loss: 1.9929
Epoch 2/5
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 1711s 927ms/step - accuracy: 0.4679 - loss: 1.7696 - val_accuracy: 0.4690 - val_loss: 1.8139
Epoch 3/5
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 1688s 931ms/step - accuracy: 0.5180 - loss: 1.5888 - val_accuracy: 0.4925 - val_loss: 1.7222
Epoch 4/5
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 1724s 943ms/step - accuracy: 0.5500 - loss: 1.4758 - val_accuracy: 0.5095 - val_loss: 1.6712
Epoch 5/5
1813/1813 ━━━━━━━━━━━━━━━━━━━━ 1730s 926ms/step - accuracy: 0.5692 - loss: 1.4040 - val_accuracy: 0.5212 - val_loss: 1.6390


In [36]:
val_loss = network_history.history['val_loss'][-1]  # final val loss
perplexity = np.exp(val_loss)
print(f"Validation Perplexity: {perplexity:.2f}")

Validation Perplexity: 5.15


## 6. Generation Criteria
- After training, generate **two distinct text samples**, each at least **50 tokens**.
- Use **different seed phrases** (e.g., “love is” vs. “time will”).

In [10]:
# convert ids to chars function
ids_to_chars = tf.keras.layers.StringLookup(vocabulary=chars_to_ids.get_vocabulary(), invert=True)

In [35]:
seed_texts = ["love is", "time will"]
for seed_text in seed_texts:
  generated = seed_text
  # generate 50 tokens
  for _ in range(50):
    input_chars = tf.strings.unicode_split(generated, 'UTF-8')  # converts to unicode chars
    input_ids = chars_to_ids(input_chars)  # converts chars to ids
    input_ids = tf.expand_dims(input_ids, 0)  # makes a batch of sequences for prediction
    predictions = model.predict(input_ids,verbose=0)  # predict character
    predicted = predictions[0]
    predicted_char_index = np.random.choice(range(len(predicted)), p=predicted)  # select character based on prob
    predicted_char = ids_to_chars([predicted_char_index]).numpy()[0].decode('utf-8')  # converts ids to char

    generated += predicted_char  # concat to generated text

  print("Generated with seed \"", seed_text, "\": ", generated)

Generated with seed " love is ":  love is we head at the car caliceuriam buling him a mils 
Generated with seed " time will ":  time will and leaving lites a short he had tame will with a
